In [53]:
# import pyspark modules
import os
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql import types as typ     # for datatype conversion
from pyspark.sql import functions as F  # for col() function
from pyspark.ml.linalg import DenseVector
from pyspark.ml.feature import StandardScaler
from pyspark.ml.regression import LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sc = SparkContext.getOrCreate()
sqlCtx = SQLContext(sc)

In [ ]:
#comment

# Data Import and Preprocessing

## Imputing, Bining and Filtering

In [2]:
suicide = pd.read_csv ('master.csv')

In [3]:
suicide.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [4]:
df = sqlCtx.createDataFrame(suicide)

In [5]:
df.show(5)

+-------+----+------+-----------+-----------+----------+-----------------+------------+------------+------------------+------------------+---------------+
|country|year|   sex|        age|suicides_no|population|suicides/100k pop|country-year|HDI for year| gdp_for_year ($) |gdp_per_capita ($)|     generation|
+-------+----+------+-----------+-----------+----------+-----------------+------------+------------+------------------+------------------+---------------+
|Albania|1987|  male|15-24 years|         21|    312900|             6.71| Albania1987|         NaN|     2,156,624,900|               796|   Generation X|
|Albania|1987|  male|35-54 years|         16|    308000|             5.19| Albania1987|         NaN|     2,156,624,900|               796|         Silent|
|Albania|1987|female|15-24 years|         14|    289700|             4.83| Albania1987|         NaN|     2,156,624,900|               796|   Generation X|
|Albania|1987|  male|  75+ years|          1|     21800|             4

In [6]:
df.count()

27820

In [7]:
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- year: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- suicides_no: long (nullable = true)
 |-- population: long (nullable = true)
 |-- suicides/100k pop: double (nullable = true)
 |-- country-year: string (nullable = true)
 |-- HDI for year: double (nullable = true)
 |--  gdp_for_year ($) : string (nullable = true)
 |-- gdp_per_capita ($): long (nullable = true)
 |-- generation: string (nullable = true)



In [8]:
df.cache()

DataFrame[country: string, year: bigint, sex: string, age: string, suicides_no: bigint, population: bigint, suicides/100k pop: double, country-year: string, HDI for year: double,  gdp_for_year ($) : string, gdp_per_capita ($): bigint, generation: string]

## outlier and missing value treatment

## feature engineering

In [9]:
import pyspark.mllib.stat as st
import numpy as np
from pyspark.sql.functions import col, asc

In [30]:
#feature transform: grouping countries into regions
Africa = ["Algeria", "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cameroon","Cabo Verde", "Cape Verde",
          "Central African Republic", "Chad", "Comoros", "Côte d’Ivoire", "Democratic Republic of the Congo",
          "Equatorial Guinea", "Eritrea", "Ethiopia", "Gabon", "Gambia", "Ghana", "Guinea", "Guinea-Bissau",
          "Kenya", "Lesotho", "Liberia", "Madagascar", "Malawi", "Mali", "Mauritania", "Mauritius", "Mozambique",
          "Namibia", "Niger", "Nigeria", "Republic of the Congo", "Rwanda", "Sao Tome and Principe", "Senegal", 
          "Seychelles", "Sierra Leone", "South Africa", "Swaziland", "Togo", "Uganda", "United Republic of Tanzania",
          "Zambia", "Zimbabwe"]
Americas = ["Antigua and Barbuda", "Aruba","Argentina", "Bahamas", "Barbados", "Belize", "Bolivia (Plurinational State of)", 
            "Brazil", "Canada", "Chile", "Colombia", "Costa Rica", "Cuba", "Dominica", "Dominican Republic", "Ecuador",
            "El Salvador", "Grenada", "Guatemala", "Guyana", "Haiti", "Honduras", "Jamaica", "Mexico", "Nicaragua", "Panama",
            "Paraguay", "Peru","Puerto Rico", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and Grenadines", "Suriname", 
            "Trinidad and Tobago", "United States", "Uruguay", "Venezuela (Bolivarian Republic of)"]
SouthEastAsia = ["Bangladesh", "Bhutan", "Democratic People’s Republic of Korea", "India", "Indonesia", "Maldives", "Myanmar",
                 "Nepal", "Sri Lanka", "Thailand", "Timor-Leste"]
European = ["Albania", "Andorra", "Armenia", "Austria", "Azerbaijan", "Belarus", "Belgium", "Bosnia and Herzegovina", "Bulgaria",
            "Croatia", "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Georgia", "Germany", "Greece", 
            "Hungary", "Iceland", "Ireland", "Israel", "Italy", "Kazakhstan", "Kyrgyzstan", "Latvia", "Lithuania", "Luxembourg",
            "Malta", "Monaco", "Montenegro", "Netherlands", "Norway", "Poland", "Portugal", "Republic of Moldova", "Romania",
            "Russian Federation", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", "Sweden", "Switzerland", "Tajikistan",
            "The former Yugoslav Republic of Macedonia", "Turkey", "Turkmenistan", "Ukraine", "United Kingdom", "Uzbekistan"]
EasternMediterranean = ["Afghanistan", "Bahrain", "Djibouti", "Egypt", "Iran (Islamic Republic of)", "Iraq", "Jordan", 
                        "Kuwait", "Lebanon", "Libyan Arab Jamahiriya", "Morocco", "Oman", "Pakistan", "Qatar", "Saudi Arabia",
                        "Somalia", "Sudan", "Syrian Arab Republic", "Tunisia", "United Arab Emirates", "Yemen"]
WesternPacific = ["Australia", "Brunei Darussalam", "Cambodia", "China", "Cook Islands", "Fiji", "Japan", "Kiribati", 
                  "Lao People’s Democratic Republic", "Malaysia", "Marshall Islands", "Micronesia (Federated States of)", 
                  "Mongolia", "Nauru", "New Zealand", "Niue", "Palau", "Papua New Guinea", "Philippines", "Republic of Korea",
                  "Samoa", "Singapore", "Solomon Islands", "Taiwan", "Tonga", "Tuvalu", "Vanuatu", "Vietnam","Macau"]

In [31]:
#filteredDF = df.filter(col("age") == "35-54 years").sort(asc("year"))

In [94]:
transformedDF = df.withColumn('region', F.when(F.col('country').isin(Africa), 'Africa')\
.when(F.col('country').isin(Americas), 'Americas') \
.when(F.col('country').isin(SouthEastAsia), 'SouthEastAsia')\
.when(F.col('country').isin(European), 'European')\
.when(F.col('country').isin(WesternPacific ), 'WesternPacific ')\
.when(F.col('country').isin(EasternMediterranean), 'EasternMediterranean'))                                                    

In [95]:
transformedDF2 = transformedDF.withColumn('year_span', F.when(((F.col("year")>=1985) & (F.col('year')<= 1988)),'1985-1988')\
.when(((F.col("year")>=1989) & (F.col('year')<= 1992)),'1989-1992')\
.when(((F.col('year')>=1993) & (F.col('year')<=1996)), '1993-1996')\
.when(((F.col('year')>=1997) & (F.col('year')<=2000)), '1997-2000')\
.when(((F.col('year')>=2001) & (F.col('year')<=2004)), '2001-2004')\
.when(((F.col('year')>=2005) & (F.col('year')<=2008)), '2005-2008')\
.when(((F.col('year')>=2009) & (F.col('year')<=2012)), '2009-2012')\
.when(((F.col('year')>=2013) & (F.col('year')<=2016)), '2013-2016'))
                                                                                    

In [96]:
df_processed = transformedDF2.withColumn("age_encoded", F.when(col("age")=='5-14 years', 0)
                                         .when(col("age")=='15-24 years', 1)
                                         .when(col("age")=='25-34 years', 2)
                                         .when(col("age")=='35-54 years', 3)
                                         .when(col("age")=='55-74 years', 4)
                                         .when(col("age")=='75+ years', 5))

In [97]:
df_processed2 = df_processed.withColumn("sex_encoded", F.when(col("sex")=='female', 0)
                                         .when(col("sex")=='male', 1))

In [107]:
#rename
df_processed2 = df_processed2.withColumnRenamed("suicides/100k pop", "suicide_100k_pop") \
.withColumnRenamed("gdp_per_capita ($)", "gdp_per_capita")

In [108]:
df_processed2.printSchema()

root
 |-- country: string (nullable = true)
 |-- year: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- suicides_no: long (nullable = true)
 |-- population: long (nullable = true)
 |-- suicide_100k_pop: double (nullable = true)
 |-- country-year: string (nullable = true)
 |-- HDI for year: double (nullable = true)
 |--  gdp_for_year ($) : string (nullable = true)
 |-- gdp_per_capita: long (nullable = true)
 |-- generation: string (nullable = true)
 |-- region: string (nullable = true)
 |-- year_span: string (nullable = true)
 |-- age_encoded: integer (nullable = true)
 |-- sex_encoded: integer (nullable = true)



In [109]:
df_trimmed = df_processed2.select(["region",
                "year_span",
                "sex_encoded",
                "age_encoded",
                "suicide_100k_pop",                              
                "gdp_per_capita", 
                "generation"])

In [70]:
df_trimmed.printSchema()

root
 |-- region: string (nullable = true)
 |-- year_span: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- sex_encoded: integer (nullable = true)
 |-- age: string (nullable = true)
 |-- age_encoded: integer (nullable = true)
 |-- suicides/100k pop: double (nullable = true)
 |-- gdp_per_capita ($): long (nullable = true)
 |-- generation: string (nullable = true)



# Exploratory data analysis

In [16]:
#Describe major attributions
df.describe(['suicides_no','population','suicides/100k pop', 'gdp_per_capita ($)',' gdp_for_year ($) ']).show()
    #suicide/100k pop is optimal than suicides_no
    #problem with the data type of gdp_for_year

+-------+------------------+------------------+------------------+------------------+------------------+
|summary|       suicides_no|        population| suicides/100k pop|gdp_per_capita ($)| gdp_for_year ($) |
+-------+------------------+------------------+------------------+------------------+------------------+
|  count|             27820|             27820|             27820|             27820|             27820|
|   mean|242.57440690150972|1844793.6173975556|12.816097411933846|16866.464414090584|              null|
| stddev| 902.0479168336386|3911779.4417563654|18.961511014503152|18887.576472205557|              null|
|    min|                 0|               278|               0.0|               251| 1,002,219,052,968|
|    max|             22338|          43805214|            224.97|            126352|       997,007,926|
+-------+------------------+------------------+------------------+------------------+------------------+



In [19]:
#compute correlation
df.corr('suicides/100k pop','gdp_per_capita ($)')

0.0017851337973438972

In [20]:
# compute correlation between all numeric features
features_numeric = ['suicides_no','population','suicides/100k pop', 'gdp_per_capita ($)']

n_numeric = len(features_numeric)
corr = []

for i in range(0, n_numeric):
 temp = [None] * i

 for j in range(i, n_numeric):
     temp.append(df.corr(features_numeric[i], features_numeric[j]))
 corr.append(temp)

In [21]:
corr
   #suicides_no and population

[[1.0, 0.6161622675219295, 0.3066044512677847, 0.06132974884024606],
 [None, 1.0, 0.008284973053478351, 0.08150985822280572],
 [None, None, 1.0, 0.0017851337973438972],
 [None, None, None, 1.0]]

In [22]:
#age group for the year span from xxxx to xxxx  -carrie
#world map heat map by year (compare)    -leyi
#group by age group and select a specific country to get a trend plot \time and population  -bryan
#correlation heatmap

### Visualization

In [23]:
#world map heat map by year (compare)    -leyi

In [24]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [80]:
temp = df_trimmed.groupby('region').sum()
temp.printSchema()

root
 |-- region: string (nullable = true)
 |-- sum(sex_encoded): long (nullable = true)
 |-- sum(age_encoded): long (nullable = true)
 |-- sum(suicides/100k pop): double (nullable = true)
 |-- sum(gdp_per_capita ($)): long (nullable = true)



In [82]:
#plt.figure(figsize=(14,6))
#df_trimmed.groupby('region').sum().plot(kind='bar',figsize=(16,8),title='Suicide Based in Regions')

plt.bar(temp['region'], temp["suicides/100k pop"], width=2000)
plt.title('Suicide Based in Regions')

AnalysisException: 'Cannot resolve column name "suicides/100k pop" among (region, sum(sex_encoded), sum(age_encoded), sum(suicides/100k pop), sum(gdp_per_capita ($)));'

In [88]:
p = pd.crosstab(index=df_trimmed.region,columns=df_trimmed.year_span,values=df.suicides_no,aggfunc='sum')

sns.heatmap(p.loc[:,2011:2015].sort_values(2015, ascending=False).dropna().head(5),annot=True)

pp = pd.crosstab(index=df.country,columns=df.year,values=df.population,aggfunc='sum')

pdivpp = p/pp*10000

ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

# Model Construction

### Main Research Question:

preprocessing:
dummy variable (sex)
missing values(HDI) 
dimensionality reduction PCA

data splitting: 
train 0.8
test 0.2

model building:
--- Supervised Learning
Random forests and gradient-boosted trees (level of significance) --leyi
Logistic Regression (includes feature scaling)  --project3 --carrie

--- Unsupervised Learning
K-mean clustering: Country & Age (generation)  --bryan


model checking:
k-fold cross validation
for regression: MSE
R-square, adjusted R-square
for classification (accuracy, precision, confusion matrix)

what we found:
(answer research question)

Research Questions:
What factor contributes the most to the high suicide rate  of both developed and developing contries and what is the global trend and pattern of the rate of suicidal accident for the past decades?
--age group
--gender
--gdp (developed and developing)
--time span(generation)

We will construct a model to rank-order the suicide rate of world's regions for people in their middle age (55-74) with a given time span (3-year-interval between 1985-2015)

## Supervised Learning--- Random Forest

### 1. Data Preprocessing

### 2. Random Forest Implementation
#### evaluate the high risk groups for suicide and identify or discriminate high-risk subjects

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorIndexer, StringIndexer, IndexToString
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
#RFmodel = RandomForest.trainRegressor(train_data, catgoricalFeaturesInfo= {}, numTrees= 1000, featureSubsetStrategy='auto', \
                                      impurity='variance', maxDepth= 4, maxBins =32)

In [ ]:
#fit the dataset to include all labels in index
labelIndexer = StringIndexer(inputCol='label', outputCol='indexedFeatures', maxCategories= 4).fit(transformedDF_trimmed)

In [ ]:
#identify categorical features and index them
featureIndexer = VectorIndexer(inputCol='features', outputCol='indexedFeatures', maxCategories= 4).fit(transformedDF_trimmed)

### 3. Data splitting

In [86]:
train_data, test_data = transformedDF_trimmed.randomSplit([0.8, 0.2], seed=123)

In [87]:
train_data, test_data.cache()

(DataFrame[country: string, year: bigint, sex: string, age: string, suicides_no: bigint, population: bigint, suicides/100k pop: double, country-year: string, HDI for year: double,  gdp_for_year ($) : string, gdp_per_capita ($): bigint, generation: string, Region: string],
 DataFrame[country: string, year: bigint, sex: string, age: string, suicides_no: bigint, population: bigint, suicides/100k pop: double, country-year: string, HDI for year: double,  gdp_for_year ($) : string, gdp_per_capita ($): bigint, generation: string, Region: string])

In [ ]:
#train a random forest model
RFmodel = RandomForestClassifier(labelCol='indexedLabel', outputCol='indexedFeatures', numTrees=10)

In [ ]:
#Convert indexed labels back to original labels
labelConverter = IndexToString(inputCol='features', outputCol='indexedFeatures', maxCategories= 4)

In [ ]:
#Chain indexers and forest in a pipeline
pipeline = Pipeline

### 4. K-fold cross validation

# Model Evaluation

## Classification--- Random Forest
- confusion matrix
- AUROC
- precision and accuracy

## References
WHO regions wikipedia